In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline 
import re

#nlp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer


from scipy.sparse import hstack
from scipy import sparse

from sklearn.linear_model import Ridge
from sklearn import linear_model

from scipy import math, stats

# Loading data and cleaning data

In [2]:
#loading data
train = pd.read_table("/Users/Jpeteyy/PythonScripts/train.tsv", encoding = 'utf-8')
test = pd.read_table("/Users/Jpeteyy/PythonScripts/test.tsv", encoding = 'utf-8')
#test this later 48952

In [3]:
#splitting categories into multiple buckets
#train['cat1'], train['cat2'], train['cat3'] = train['category_name'].str.split('/', 2).str
for p,q in enumerate(['cat1','cat2','cat3']):
    train[q] = train['category_name'].apply(lambda x: x.split('/')[p] if pd.isnull(x) != True else '')
    
    
#splitting categories into multiple buckets
#train['cat1'], train['cat2'], train['cat3'] = train['category_name'].str.split('/', 2).str
for p,q in enumerate(['cat1','cat2','cat3']):
    test[q] = test['category_name'].apply(lambda x: x.split('/')[p] if pd.isnull(x) != True else '')

In [4]:
for k in ['item_description', 'name']:
    train[k] = train[k].apply(lambda x: str.lower(x) if type(x) == str else '')
    
    
for k in ['item_description', 'name']:
    test[k] = test[k].apply(lambda x: str.lower(x) if type(x) == str else '')

In [5]:
#replacing missing values function
def replace_nan(dataframe):
    #dataframe.category_name.fillna(value="missing", inplace=True)
    dataframe.brand_name.fillna(value="missing", inplace=True)
    #dataframe.item_description.fillna(value="missing", inplace=True)
    #dataframe.cat1.fillna(value="missing", inplace=True)
    #dataframe.cat2.fillna(value="missing", inplace=True)
    #dataframe.cat3.fillna(value="missing", inplace=True)
    #dataframe.cat4.fillna(value="missing", inplace=True)
    return (dataframe)

In [6]:
#passing training data into replacing missing func
train = replace_nan(train)

test = replace_nan(test)

In [7]:
train.head(2)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3
0,0,mlb cincinnati reds t shirt size xl,3,Men/Tops/T-shirts,missing,10.0,1,no description yet,Men,Tops,T-shirts
1,1,razer blackwidow chroma keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,this keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts


In [8]:
#cleaning up string fields


#item_description
train['item_description'] = train.item_description.str.lower()
train['item_description'] = train.item_description.replace('[', ' ')
train['item_description'] = train.item_description.replace(']', ' ')
train['item_description'] = train.item_description.replace('[^0-9a-zA-Z]+', ' ', regex = True)
train['item_description'] = train.item_description.replace(" rm ", '', regex = True)
train['item_description'] = train.item_description.replace('[^a-zA-Z0-9 ]', '', regex = True)


#brand_names
train['brand_name'] = train.brand_name.str.lower()
train['brand_name'] = train.brand_name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_names
train['name'] = train.name.str.lower()
train['name'] = train.name.replace('[', ' ')
train['name'] = train.name.replace(']', ' ')
train['name'] = train.name.replace('[^0-9a-zA-Z]+', ' ', regex = True)
train['name'] = train.name.replace(' rm ', '')
#train['name'] = train.name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_condition_id
train['item_condition_id'] = train.item_condition_id.replace('5','4')


#categories
train['category_name'] = train.category_name.str.lower()
train['cat1'] = train.cat1.str.lower()
train['cat2'] = train.cat2.str.lower()
train['cat3'] = train.cat3.str.lower()
#train['cat4'] = train.cat4.str.lower()

In [9]:
#cleaning up string fields


#item_description
test['item_description'] = test.item_description.str.lower()
test['item_description'] = test.item_description.replace('[', ' ')
test['item_description'] = test.item_description.replace(']', ' ')
test['item_description'] = test.item_description.replace('[^0-9a-zA-Z]+', ' ', regex = True)
test['item_description'] = test.item_description.replace(" rm ", '', regex = True)
test['item_description'] = test.item_description.replace('[^a-zA-Z0-9 ]', '', regex = True)


#brand_names
test['brand_name'] = test.brand_name.str.lower()
test['brand_name'] = test.brand_name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_names
test['name'] = test.name.str.lower()
test['name'] = test.name.replace('[', ' ')
test['name'] = test.name.replace(']', ' ')
test['name'] = test.name.replace('[^0-9a-zA-Z]+', ' ', regex = True)
test['name'] = test.name.replace(' rm ', '')
#train['name'] = train.name.replace('[^a-zA-Z0-9 ]', '', regex = True)


#item_condition_id
test['item_condition_id'] = test.item_condition_id.replace('5','4')


#categories
test['category_name'] = test.category_name.str.lower()
test['cat1'] = test.cat1.str.lower()
test['cat2'] = test.cat2.str.lower()
test['cat3'] = test.cat3.str.lower()
#train['cat4'] = train.cat4.str.lower()

In [10]:
train.head(2)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,cat1,cat2,cat3
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,missing,10.0,1,no description yet,men,tops,t-shirts
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,razer,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts


# Extracting additional brands

In [11]:
#O(n) string search to extract brands
import ahocorasick

In [12]:
def string_search(dictionary, listStrings, listSubstrings):
    auto = ahocorasick.Automaton()
    for substr in listSubstrings:
        auto.add_word(substr, substr)
    auto.make_automaton()
    
    for idx, astr in enumerate(listStrings):
        seen = set()
        match_count = 0
        for end_ind, found in auto.iter(astr):
            if (found not in seen) and (match_count == 0):
                seen.add(found)
                #print(idx)
                #print(found+ "   " + astr)
                dictionary.update({idx: found})
                match_count +=1
#https://stackoverflow.com/questions/34816775/python-optimal-search-for-substring-in-list-of-strings
#http://ieva.rocks/2016/11/24/keyword-matching-with-aho-corasick/

In [13]:
#lookup table of missing brand names to index
missing_brand_lookup_train = pd.DataFrame(train[train['brand_name'] == 'missing'].reset_index().train_id)
missing_brand_lookup_train['join_key'] = missing_brand_lookup_train.index.values
#left join on train_id of output = index, and take the id number





#lookup table of missing brand names to index
missing_brand_lookup_test = pd.DataFrame(test[test['brand_name'] == 'missing'].reset_index().test_id)
missing_brand_lookup_test['join_key'] = missing_brand_lookup_test.index.values
#left join on train_id of output = index, and take the id number

In [14]:
#missing brands in the item name
name_strings_train =  train[train['brand_name'] ==
                        'missing'].reset_index().name.tolist() 
                        #string names where brand_name is null
name_strings_train = [item + ' ' for item in name_strings_train] # add ' ' to prevent missed brands at end of word



#missing brands in the description
desc_strings_train =  train[train['brand_name'] ==
                        'missing'].reset_index().item_description.tolist() 
                        #string names where brand_name is null
desc_strings_train = [item + ' ' for item in desc_strings_train] # add ' ' to prevent missed brands at end of word




#building list of existing brands
brand_substrings_train = train[train['brand_name'] != 
                        'missing'].reset_index().brand_name.tolist() 
                        #list of brand names where brand_name not null
    
brand_substrings_train = list(set(brand_substrings_train)) #de-duping list of brands

brand_substrings_train = [s for s in brand_substrings_train if len(s) > 2] #restricting brand substring length to > 2

if "all" in brand_substrings_train: brand_substrings_train.remove("all") #removing 'all' brand b/c too common of word
if "and" in brand_substrings_train: brand_substrings_train.remove("and") #removing 'all' brand b/c too common of word
if "pink" in brand_substrings_train: brand_substrings_train.remove("pink") #removing 'all' brand b/c too common of word
    
brand_substrings_train.sort(key=len, reverse=True) #sorting on longest string first

brand_substrings_train =  [item + ' ' for item in brand_substrings_train] #add ' ' to prevent misclassifications in middle of words





#initializing dictionaries to collect extracted brand names

brands_from_name_field_train = {}

brands_from_description_field_train = {}





In [15]:
#missing brands in the item name
name_strings_test =  test[test['brand_name'] ==
                        'missing'].reset_index().name.tolist() 
                        #string names where brand_name is null
name_strings_test = [item + ' ' for item in name_strings_test] # add ' ' to prevent missed brands at end of word



#missing brands in the description
desc_strings_test =  test[test['brand_name'] ==
                        'missing'].reset_index().item_description.tolist() 
                        #string names where brand_name is null
desc_strings_test = [item + ' ' for item in desc_strings_test] # add ' ' to prevent missed brands at end of word




#building list of existing brands
brand_substrings_test = test[test['brand_name'] != 
                        'missing'].reset_index().brand_name.tolist() 
                        #list of brand names where brand_name not null
    
brand_substrings_test = list(set(brand_substrings_test)) #de-duping list of brands

brand_substrings_test = [s for s in brand_substrings_test if len(s) > 2] #restricting brand substring length to > 2

if "all" in brand_substrings_test: brand_substrings_test.remove("all") #removing 'all' brand b/c too common of word
if "and" in brand_substrings_test: brand_substrings_test.remove("and") #removing 'all' brand b/c too common of word
if "pink" in brand_substrings_test: brand_substrings_test.remove("pink") #removing 'all' brand b/c too common of word
    
brand_substrings_test.sort(key=len, reverse=True) #sorting on longest string first

brand_substrings_test =  [item + ' ' for item in brand_substrings_test] #add ' ' to prevent misclassifications in middle of words





#initializing dictionaries to collect extracted brand names

brands_from_name_field_test = {}

brands_from_description_field_test = {}





In [16]:
#counting number of previous missing brands
previous_brand_count = train[train["brand_name"] != 'missing'].shape[0]
print("There are " + str(previous_brand_count) + " brands present in the training data")

There are 849853 brands present in the training data


In [17]:
print(previous_brand_count / train.shape[0])

0.5732431274809701


# brand extraction from item name

In [18]:
#brands hiding in name
string_search(brands_from_name_field_train, name_strings_train, brand_substrings_train)


#brands hiding in name
string_search(brands_from_name_field_test, name_strings_test, brand_substrings_test)

In [19]:
extracted_brands_train = pd.DataFrame(list(brands_from_name_field_train.items()))

extracted_brands_test = pd.DataFrame(list(brands_from_name_field_test.items()))

In [20]:
extracted_brands_train.columns = ['train_id', 'brand_name']

extracted_brands_train['brand_name'] = extracted_brands_train['brand_name'].map(lambda x: str(x)[:-1])

extracted_brands_test.columns = ['test_id', 'brand_name']

extracted_brands_test['brand_name'] = extracted_brands_test['brand_name'].map(lambda x: str(x)[:-1])

In [21]:
extracted_brands_train = pd.merge(extracted_brands_train,missing_brand_lookup_train, left_on='train_id', right_on='join_key', how='left')


extracted_brands_test = pd.merge(extracted_brands_test,missing_brand_lookup_test, left_on='test_id', right_on='join_key', how='left')

In [22]:
train = pd.merge(train, extracted_brands_train, how='left', left_on='train_id', right_on='train_id_y')
train.brand_name_y = train.brand_name_y.fillna('missing')

test = pd.merge(test, extracted_brands_test, how='left', left_on='test_id', right_on='test_id_y')
test.brand_name_y = test.brand_name_y.fillna('missing')

In [23]:
train['brand_name'] = train[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)


test['brand_name'] = test[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)

In [24]:
train.drop('brand_name_x', axis=1, inplace=True)
train.drop('brand_name_y', axis=1, inplace=True)
train.drop('train_id_x', axis=1, inplace=True)
train.drop('train_id_y', axis=1, inplace=True)
train.drop('join_key', axis=1, inplace=True)


test.drop('brand_name_x', axis=1, inplace=True)
test.drop('brand_name_y', axis=1, inplace=True)
test.drop('test_id_x', axis=1, inplace=True)
test.drop('test_id_y', axis=1, inplace=True)
test.drop('join_key', axis=1, inplace=True)

In [25]:
train.tail(2)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name
1482533,1482533,world markets lanterns,3,home/home décor/home décor accents,45.0,1,there is 2 of each one that you see so 2 red 2...,home,home décor,home décor accents,missing
1482534,1482534,brand new lux de ville wallet,1,women/women's accessories/wallets,22.0,0,new with tag red with sparkle firm price no fr...,women,women's accessories,wallets,lux


In [26]:
print("Extracted " + str(train[train["brand_name"] != 'missing'].shape[0]  - previous_brand_count) + " brands from item name in training data")
previous_brand_count = train[train["brand_name"] != 'missing'].shape[0]

Extracted 278179 brands from item name in training data


# brand extraction from item description

In [27]:
#brands hiding in description
string_search(brands_from_description_field_train, desc_strings_train, brand_substrings_train)


#brands hiding in description
string_search(brands_from_description_field_test, desc_strings_test, brand_substrings_test)

In [28]:
extracted_brands_train = pd.DataFrame(list(brands_from_description_field_train.items()))
extracted_brands_train.columns = ['train_id', 'brand_name']

extracted_brands_train['brand_name'] = extracted_brands_train['brand_name'].map(lambda x: str(x)[:-1])
extracted_brands_train = pd.merge(extracted_brands_train,missing_brand_lookup_train, left_on='train_id', right_on='join_key', how='left')


extracted_brands_test = pd.DataFrame(list(brands_from_description_field_test.items()))
extracted_brands_test.columns = ['test_id', 'brand_name']

extracted_brands_test['brand_name'] = extracted_brands_test['brand_name'].map(lambda x: str(x)[:-1])
extracted_brands_test = pd.merge(extracted_brands_test,missing_brand_lookup_test, left_on='test_id', right_on='join_key', how='left')

In [29]:
train = pd.merge(train, extracted_brands_train, how='left', left_on='train_id', right_on='train_id_y')
train.brand_name_y = train.brand_name_y.fillna('missing')

train['brand_name'] = train[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)
#testing
#train[(train['brand_name_x'] == 'missing') & (train['brand_name'] != 'missing')]




test = pd.merge(test, extracted_brands_test, how='left', left_on='test_id', right_on='test_id_y')
test.brand_name_y = test.brand_name_y.fillna('missing')

test['brand_name'] = test[['brand_name_x','brand_name_y']].apply(
    lambda x: x['brand_name_y'] 
    if x['brand_name_x'] == 'missing' 
    else x['brand_name_x'], axis=1)
#testing
#train[(train['brand_name_x'] == 'missing') & (train['brand_name'] != 'missing')]

In [30]:
train.drop('brand_name_x', axis=1, inplace=True)
train.drop('brand_name_y', axis=1, inplace=True)
train.drop('train_id_x', axis=1, inplace=True)
train.drop('train_id_y', axis=1, inplace=True)
train.drop('join_key', axis=1, inplace=True)

test.drop('brand_name_x', axis=1, inplace=True)
test.drop('brand_name_y', axis=1, inplace=True)
test.drop('test_id_x', axis=1, inplace=True)
test.drop('test_id_y', axis=1, inplace=True)
test.drop('join_key', axis=1, inplace=True)

In [31]:
test.tail(2)

,test_id,name,item_condition_id,category_name,shipping,item_description,cat1,cat2,cat3,brand_name
693357,693357,hi low floral kimono,2,women/swimwear/cover-ups,0,floral kimono tropical print open front hi low...,women,swimwear,cover-ups,ring
693358,693358,freeship 2 floral scrub tops medium,2,women/tops & blouses/t-shirts,1,2 floral scrub tops worn less than 5 times eac...,women,tops & blouses,t-shirts,missing


In [32]:
print("Extracted " + str(train[train["brand_name"] != 'missing'].shape[0]  - previous_brand_count) + " brands from item desc in training data")

Extracted 184545 brands from item desc in training data


In [33]:
print(train[train.brand_name != 'missing'].shape[0] / train.shape[0])

0.8853598734599858


# Grouping data by category and brand

In [34]:
train['brand_name'] = train.brand_name.replace('missing', np.nan)

brand_summary = train.groupby(['cat1','cat2','cat3','brand_name']).agg({'count'})['price']

In [35]:
#brand_summary.head(50)

In [36]:
#renaming columns
brand_summary.rename(columns={'count':'brand_item_count'}, inplace=True)

brand_summary = brand_summary[brand_summary.brand_item_count>100].\
                sort_values(by='brand_item_count', ascending=False).reset_index().copy()

In [37]:
brand_summary.tail(5)

,cat1,cat2,cat3,brand_name,brand_item_count
1812,women,dresses,knee-length,que,101
1813,women,jewelry,earrings,silpada,101
1814,women,women's accessories,sunglasses,chanel,101
1815,women,swimwear,two-piece,ping,101
1816,handmade,bags and purses,wristlet,vera bradley,101


In [38]:
#generating random model names for each category
brand_model_num = pd.Series(['brand_'+str(np.random.randint(1,len(brand_summary)*10000)) for i in 
                            range(len(brand_summary))],name='brand_model_num',dtype=str)

brand_summary = pd.concat([brand_summary,brand_model_num], axis=1)

brand_summary.head()

,cat1,cat2,cat3,brand_name,brand_item_count,brand_model_num
0,women,athletic apparel,"pants, tights, leggings",lularoe,26184,brand_13453330
1,women,underwear,bras,victorias secret,10622,brand_16494801
2,electronics,video games & consoles,games,nintendo,10517,brand_5713935
3,women,tops & blouses,t-shirts,pink,9307,brand_4991520
4,women,shoes,athletic,nike,8108,brand_17466286


In [39]:
train= pd.merge(train,brand_summary,how='left',on=['cat1','cat2','cat3','brand_name'])



test= pd.merge(test,brand_summary,how='left',on=['cat1','cat2','cat3','brand_name'])

In [40]:
#train.head(20)

In [41]:
#looking at train dataset with categories

In [42]:
#simple look at train dataset with categories
train.loc[train.brand_model_num != None, ['cat1','cat2','cat3','brand_name','brand_model_num','price']].\
groupby(['cat1','cat2','cat3','brand_name','brand_model_num']).agg({pd.Series.count})['price'].\
reset_index().sort_values(by='count', ascending=False).head(5)

,cat1,cat2,cat3,brand_name,brand_model_num,count
1056,women,athletic apparel,"pants, tights, leggings",lularoe,brand_13453330,26184.0
1686,women,underwear,bras,victorias secret,brand_16494801,10622.0
434,electronics,video games & consoles,games,nintendo,brand_5713935,10517.0
1612,women,tops & blouses,t-shirts,pink,brand_4991520,9307.0
1357,women,shoes,athletic,nike,brand_17466286,8108.0


In [43]:
#uncategorized items, grouping at category level, where count of items >  100

In [44]:
cat3_summary = train[pd.isnull(train.brand_model_num) == True].groupby(['cat1','cat2','cat3'])\
                .agg({'count'})['price']
    
cat3_summary.rename(columns={'count':'cat3_item_count'},inplace = True)

cat3_summary = cat3_summary[cat3_summary.cat3_item_count>100].\
                sort_values(by='cat3_item_count',ascending=False).reset_index().copy()

In [45]:
#generating random model names for each category

cat3_model_num = pd.Series(['cat3_'+str(np.random.randint(1,len(cat3_summary)*10000)) for i in
                           range(len(cat3_summary))], name='cat3_model_num',dtype = str)

cat3_summary = pd.concat([cat3_summary.reset_index(),cat3_model_num], axis = 1)

In [46]:
cat3_summary.head()

,index,cat1,cat2,cat3,cat3_item_count,cat3_model_num
0,0,women,tops & blouses,t-shirts,12905,cat3_2212835
1,1,women,tops & blouses,blouse,9854,cat3_3532825
2,2,women,dresses,"above knee, mini",9188,cat3_5910305
3,3,men,tops,t-shirts,8666,cat3_4871016
4,4,women,tops & blouses,"tank, cami",8032,cat3_5180224


In [47]:
train = pd.merge(train,cat3_summary,how='left',on=['cat1','cat2','cat3'])

test = pd.merge(test,cat3_summary, how = 'left', on = ['cat1','cat2','cat3'])

In [48]:
train['final_model_num'] = train.apply(lambda x: x['cat3_model_num'] if pd.isnull(x['brand_model_num']) == True
                                      else x['brand_model_num'], axis = 1)


test['final_model_num'] = test.apply(lambda x: x['cat3_model_num'] if pd.isnull(x['brand_model_num']) == True
                                      else x['brand_model_num'], axis = 1)

In [49]:
train['final_item_count'] = train.apply(lambda x: x['cat3_item_count'] if pd.isnull(x['brand_item_count']) == True
                                       else x['brand_item_count'], axis = 1)


train.drop(['brand_item_count','brand_model_num','index', 'cat3_item_count','cat3_model_num'], axis = 1, inplace = True)



test['final_item_count'] = test.apply(lambda x: x['cat3_item_count'] if pd.isnull(x['brand_item_count']) == True
                                       else x['brand_item_count'], axis = 1)


test.drop(['brand_item_count','brand_model_num','index', 'cat3_item_count','cat3_model_num'], axis = 1, inplace = True)

In [50]:
train.head(3)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,10.0,1,no description yet,men,tops,t-shirts,mlb,cat3_4871016,8666.0
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts,razer,cat3_1939564,807.0
2,2,ava viv blouse,1,women/tops & blouses/blouse,10.0,1,adorable top with a hint of lace and a key hol...,women,tops & blouses,blouse,target,cat3_3532825,9854.0


In [51]:
print(train.isnull().sum())

train_id                  0
name                      0
item_condition_id         0
category_name          6327
price                     0
shipping                  0
item_description          0
cat1                      0
cat2                      0
cat3                      0
brand_name           169958
final_model_num       15669
final_item_count      15669
dtype: int64


In [52]:
#train.brand_name.fillna(value = "missing", inplace = True)
#train.category_name.fillna(value="missing", inplace = True)
#train[train.brand_name == 'missing']

In [53]:
train[train.brand_name == 'victorias secret']

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
11,11,new vs pi k body mists,1,beauty/fragrance/women,34.0,0,5 new vs pink body mists 2 5 oz each fresh cl...,beauty,fragrance,women,victorias secret,brand_12139413,5694.0
21,21,nwt vs ultimate sports bra 34ddd,1,women/athletic apparel/sports bras,20.0,0,nwt victoria s secret ultimate sport bra maxim...,women,athletic apparel,sports bras,victorias secret,brand_2232551,2368.0
39,39,victoria secret 34 c corest top,3,women/underwear/bras,10.0,0,victoria secret 34 c corest top will bundle to...,women,underwear,bras,victorias secret,brand_16494801,10622.0
111,111,nwt victoria secret bathingsuit bottoms,1,women/swimwear/two-piece,26.0,0,size small nwt bathing suit bottoms,women,swimwear,two-piece,victorias secret,brand_2900447,3884.0
118,118,pink body mist,3,beauty/fragrance/women,16.0,1,bombshell eau so sexy bombshell was used a bit...,beauty,fragrance,women,victorias secret,brand_12139413,5694.0
178,178,vs pink 1 2zip collared sz m,1,women/sweaters/collared,47.0,1,price firm don t ask me for the lowest price p...,women,sweaters,collared,victorias secret,brand_13708256,107.0
182,182,victoria s secret pink night shirt,2,women/tops & blouses/other,12.0,0,from victoria s secret pink plaid and white pa...,women,tops & blouses,other,victorias secret,brand_2338929,137.0
190,190,victoria s secret small bikini top,2,women/swimwear/two-piece,9.0,0,in brand new condition just missing the strap,women,swimwear,two-piece,victorias secret,brand_2900447,3884.0
226,226,victoria s secret pj set size med,3,women/other/other,18.0,0,victoria s secret hot pink pj set with tank to...,women,other,other,victorias secret,brand_14489576,857.0
386,386,victoria s secret swim suit,3,women/swimwear/two-piece,25.0,0,good used condition,women,swimwear,two-piece,victorias secret,brand_2900447,3884.0


In [54]:
train.brand_name.fillna(value = "missing", inplace = True)
train.category_name.fillna(value="missing", inplace = True)

test.brand_name.fillna(value = "missing", inplace = True)
test.category_name.fillna(value="missing", inplace = True)

# Removing brand noise from item descriptions and names where model is at brand level

# Modeling

In [55]:
#3 part plan to improve model: 
#1 - missing models < 100 and models w no categories. model is predicting no categories, not predicting < 100
#2 - running entire model with testing data and submitting kernel.
#3 - improving poor categories with mean/median?

In [56]:
#stop word list to remove
my_stopword_list = ['and','to','the','of', 'a','are','for','you','or','it']

In [57]:
#testing mse
import math
def rmsle(y_test, y_pred):
    assert len(y_test) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y_test[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y_test))) ** 0.5

In [58]:
classifiers = [
    #svm.SVR(),
    Ridge(alpha=2, random_state=241),
    linear_model.SGDRegressor(),
    #linear_model.BayesianRidge(),
    #linear_model.LassoLars(),
    #linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    #linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()
    ]

vectorizer = TfidfVectorizer(ngram_range = (1,2), stop_words=my_stopword_list)#, max_features = 50)
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [59]:
#extracting model number where categories are null, so that we can use brands as features
model_no_cat_train = str(set(train[train.category_name == 'missing']['final_model_num'].values))

#extracting model number where categories are null, so that we can use brands as features
model_no_cat_test = str(set(test[test.category_name == 'missing']['final_model_num'].values))

In [60]:
train.head(2)

,train_id,name,item_condition_id,category_name,price,shipping,item_description,cat1,cat2,cat3,brand_name,final_model_num,final_item_count
0,0,mlb cincinnati reds t shirt size xl,3,men/tops/t-shirts,10.0,1,no description yet,men,tops,t-shirts,mlb,cat3_4871016,8666.0
1,1,razer blackwidow chroma keyboard,3,electronics/computers & tablets/components & p...,52.0,0,this keyboard is in great condition and works ...,electronics,computers & tablets,components & parts,razer,cat3_1939564,807.0


In [61]:
#['shipping2'] = train.shipping.str + 'shippingprice'

In [62]:
#train['item_condition_id'] = train.item_condition_it + 'itemconditionid'

In [ ]:
from gensim.models import word2vec

# load Google's pre-trained word2vec
import gensim

pretrained_word2vec_path = '/Users/Jpeteyy/Desktop/GoogleNews-vectors-negative300.bin'
word2vec = gensim.models.KeyedVectors.load_word2vec_format(pretrained_word2vec_path, binary=True)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [76]:
d = {1 : 'fantastic condition', 2: 'good condition', 3: 'average condition', 4: 'bad condition'}

train['condition'] = train['item_condition_id'].map(d)
test['condition'] = test['item_condition_id'].map(d)

s = {0: 'free shipping', 1: 'extra shipping'}
train['new_shipping'] = train['shipping'].map(s)
test['new_shipping'] = test['shipping'].map(s)

train['new_description'] = train['item_description'].astype(str)\
+ ' ' + train['brand_name'].map(str) + ' ' + train['condition'].map(str) + ' ' + \
train['name'].map(str) + ' ' + train['new_shipping'].map(str)

test['new_description'] = test['item_description'].astype(str)\
+ ' ' + test['brand_name'].map(str) + ' ' + test['condition'].map(str) + ' ' + \
test['name'].map(str) + ' ' + test['new_shipping'].map(str)

In [ ]:
text = train.new_description.astype(str).tolist()
#print(type(text))
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

sequences = tokenizer.texts_to_sequences(text) # list, same length as data. represent word as rank/index
padded_seq = pad_sequences(sequences)

In [72]:
#n_words = 20 # top most common words
#text = #train.item_description.astype(str), train.name.str, train.item_condition_id.apply(str),\
                  #train.brand_name.str, train.shipping.apply(str)]).tolist()
    

print('padded_seq.shape', padded_seq.shape)


word_index = tokenizer.word_index
vocab_size = len(word_index)
EMBEDDING_DIM = 300 # this is from the pretrained vectors

embedding_matrix = np.zeros((vocab_size + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec:
        embedding_vector = word2vec[word]
    else:
        embedding_vector = None
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector



import keras.backend as K

def rmsle(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    
    return K.sqrt(K.mean(K.square(first_log - second_log), axis=-1))

import keras
def truncated_normal(seed):
    return keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=seed)


from keras.layers import Embedding, Dense, Input, Flatten
from keras.layers import Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from keras.models import Model

# parameters
input_length = padded_seq.shape[1] # len (num words) of longest description
seed = 0
filter_sizes = [2,3]
n_filters = 2
dropout_prob = 0.5

def create_cnn(include_top=True, weights=None):    

    # input
    sequence_input = Input(shape=(input_length,), dtype='int32', name='input')
    #item_condition = Input(shape=[1], name="item_condition")
    #num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    # embedding_layer
    embedding_layer = Embedding(vocab_size + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=input_length
                                , name='embedding', trainable=False)(sequence_input)
    # conv layer
    features = []
    i = 0
    for filter_size in filter_sizes:
        i += 1
        # conv layer
        conv = Conv1D(n_filters, filter_size, activation='relu', kernel_initializer=truncated_normal(seed)
                      , name='conv'+str(i))(embedding_layer)
        # global max pooling
        conv = GlobalMaxPooling1D(name='pool'+str(i))(conv)
        # add features together
        features.append(conv)
    # penultimate layer
    nn = Concatenate(name='features')(features)
    if include_top:
        # dropout
        nn = Dropout(dropout_prob, seed=seed, name='dropout')(nn)
        # fully connected layer
        preds = Dense(1, kernel_initializer=truncated_normal(seed), name='output')(nn)

        model = Model(sequence_input, preds)
    else:
        model = Model(sequence_input, nn)
    
    
    if weights is not None:
        model.set_weights(weights)
        
    return model

model = create_cnn()
model.summary()

from sklearn.model_selection import KFold

x = padded_seq
y = np.asarray(train['price'])

n_splits = 2

kf = KFold(n_splits=n_splits)
kf.get_n_splits(x)


model.compile(loss='msle', optimizer='adadelta', metrics=[rmsle])

batch_size = 128
epochs = 1
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)

padded_seq.shape (1482535, 281)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 281)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 281, 300)     69969300    input[0][0]                      
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 280, 2)       1202        embedding[0][0]                  
__________________________________________________________________________________________________
conv2 (Conv1D)                  (None, 279, 2)       1802        embedding[0][0]                  
_____________________________________________________________________________

In [74]:
def rmsle(y_test, y_pred):
    assert len(y_test) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y_test[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y_test))) ** 0.5

In [78]:
model_scores = {}

test_id = []
price_prediction = []

np_array_train = train.final_model_num.unique()
np_clean_train = [x for x in np_array_train if str(x) != 'nan']

np_array_test = test.final_model_num.unique()
np_clean_test = [x for x in np_array_test if str(x) != 'nan']

for model1 in np_clean_train:
    
    vectorizer1 = TfidfVectorizer(ngram_range = (1,2), stop_words=my_stopword_list)
    vectorizer2 = TfidfVectorizer(ngram_range = (1,2), stop_words=my_stopword_list)
    if model1 == None:
        continue
    else:
        print(model1)
        train1 = train[train.final_model_num == model1]
        

        
        
        X_tfidf = vectorizer1.fit_transform(train1['item_description'].values)
        X_tfidf2 = vectorizer2.fit_transform(train1['name'].values)
        #enc = DictVectorizer()


        if (model1[:4] == 'cat3') or (model1 == model_no_cat_train):

            #adding dummy variable matrix for category items
            dummies = pd.get_dummies(train1['brand_name']).as_matrix()
            
            X = hstack([
                    #sparse.csr_matrix(dummies), 
                    X_tfidf,
                    X_tfidf2,
                    np.vstack(train1.shipping.values),
                    np.vstack(train1.item_condition_id.astype(int))
                    ])
        else:
            X = hstack([
                    X_tfidf, 
                    X_tfidf2,
                    np.vstack(train1.shipping.values),
                    np.vstack(train1.item_condition_id.astype(int))
                   ])


        y = train1.price

        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=242)

        d = {}
        for idx, item in enumerate(classifiers):
            #print(type(item))
            clf = item
            clf.fit(X_train, y_train)

            #clf = Ridge(alpha=2, random_state=241)

            ## train model on data
            #clf.fit(X_train, y_train)

            y_pred = clf.predict(X_test)

            y_pred[y_pred < 0] = 0

            score = rmsle(y_test.values,y_pred)

            c = {str(idx) : score}
            d.update(c)
            #print(score)

            #print("Brand: " + str(brand)  +  "\n Model: " +  "\n rmsle = " + str(score))

        #lowest rmsle
        key_min = min(d.keys(), key=(lambda k: d[k]))
        #print(key_min)
        
        clf = classifiers[int(key_min)]
        clf.fit(X_train, y_train)

            #clf = Ridge(alpha=2, random_state=241)

            ## train model on data
            #clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        y_pred[y_pred < 0] = 0
        
        final_score =rmsle(y_test.values,y_pred)
        #print(final_score)
        

        
        
        model_scores.update({model1 : final_score}) 
        #y['pred'] = y_pred
        
        
        if test[test.final_model_num == model1].shape[0] == 0:
            continue
        else:
            test1 = test[test.final_model_num == model1]
            
            X_tfidf = vectorizer1.transform(test1['item_description'].values)
            X_tfidf2 = vectorizer2.transform(test1['name'].values)
            
            if (model1[:4] == 'cat3') or (model1 == model_no_cat_test):
                
                #adding dummy variable matrix for category items
                #dummies = pd.get_dummies(test1['brand_name']).as_matrix()
            
                X = hstack([
                    #sparse.csr_matrix(dummies), 
                    X_tfidf,
                    X_tfidf2,
                    np.vstack(test1.shipping.values),
                    np.vstack(test1.item_condition_id.astype(int))
                    ])
            else:
                X = hstack([
                    X_tfidf, 
                    X_tfidf2,
                    np.vstack(test1.shipping.values),
                    np.vstack(test1.item_condition_id.astype(int))
                    ])
            if final_score > .5:
                data = test1.new_description.astype(str).tolist()
                sequences = tokenizer.texts_to_sequences(data) # list, same length as data. represent word as rank/index
                x_test = pad_sequences(sequences, maxlen=input_length)
                y_pred = model.predict(x_test, batch_size=batch_size)
            #print('y_test.shape', y_test.shape)
            else:
                y_pred = clf.predict(X)

            y_pred[y_pred < 0] = 0

            test1['predicted_price'] = y_pred
            
            test_id.extend(test1.test_id.values.tolist())
            price_prediction.extend(test1.predicted_price.tolist())

cat3_4871016


/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1028: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/Users/Jpeteyy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


cat3_1939564
cat3_3532825
cat3_4437720
cat3_4938684
cat3_2399287
brand_14794427
cat3_2364097
brand_7309584
cat3_4322313
brand_17946870
brand_12139413
brand_16107249
cat3_2561209
brand_6495779
brand_1508601
brand_6484
cat3_3583411
brand_11700974
brand_18002344
brand_8430246
brand_2232551
brand_13680233
cat3_4173470
brand_4538913
brand_9949246
brand_2885737
brand_13453330
brand_5049619
brand_7562587
brand_11358554
brand_16814523
brand_7916113
cat3_5494666
brand_2511048
cat3_5336073
brand_7800451
brand_7756264
brand_16494801
brand_18099670
brand_12834639
brand_1071827
cat3_3235933
brand_10807917
brand_12258723
brand_1923636
cat3_164352
brand_13566929
brand_7631162
cat3_5795225
cat3_126368
brand_1862731
brand_8528853
cat3_1734096
brand_9314821
brand_4532696
brand_12722503
cat3_2518337
brand_5599987
brand_15946378
brand_10484942
brand_9078140
brand_13196112
brand_12091598
cat3_5924471
cat3_25314
brand_15110010
brand_15719307
cat3_1844676
cat3_4156865
brand_680115
brand_7488774
brand_1127273

In [79]:
train.final_model_num.fillna(value="missing", inplace = True)

In [80]:
no_model_no_brand = train[(train.final_model_num == 'missing') & (train.brand_name == 'missing')]\
                    .groupby(['cat1','cat2','cat3'])['price'].mean().reset_index().copy()
    
no_model_no_brand.rename(columns={'price': 'median_price'}, inplace=True)

#no_model_no_brand['final_model_num_no_mod_no_brnd'] = pd.Series(['cat3_'+str(np.random.randint(1,len(no_model_no_brand)*10000)) for i in
#                           range(len(no_model_no_brand))], dtype = str)

In [81]:
no_model_w_brand = train[(train.final_model_num == 'missing') & (train.brand_name != 'missing')]\
                    .groupby(['cat1','cat2','cat3','brand_name'])['price'].mean().reset_index().copy()

no_model_w_brand.rename(columns={'price':'median_price'},inplace = True)

#no_model_w_brand ['final_model_num_no_mod_w_brnd'] = pd.Series(['cat3_'+str(np.random.randint(1,len(no_model_w_brand)*10000)) for i in
#                           range(len(no_model_w_brand))], dtype = str)

In [82]:
train = pd.merge(train, no_model_no_brand, how= 'left',on = ['cat1','cat2','cat3'])
train = pd.merge(train, no_model_w_brand, how= 'left',on = ['cat1','cat2','cat3','brand_name'])

test = pd.merge(test, no_model_no_brand, how = 'left', on = ['cat1','cat2','cat3'])
test = pd.merge(test, no_model_w_brand, how = 'left', on = ['cat1','cat2','cat3','brand_name'])

In [83]:
#train.drop(['median_price'], axis = 1, inplace = True)

In [84]:
train['median_price'] = train.apply(lambda x: x['median_price_x'] if (x['final_model_num'] == 'missing'
                                        and str(x['median_price_y']) == 'nan')
                                      else x['median_price_y']
                                        , axis = 1)

#train['median_price'] = train.median_price_x.combine_first(train.median_price_y)

train.drop(['median_price_x','median_price_y'], inplace = True, axis = 1)          


test['median_price'] = test.apply(lambda x: x['median_price_x'] if (x['final_model_num'] == 'missing'
                                        and str(x['median_price_y']) == 'nan')
                                      else x['median_price_y']
                                        , axis = 1)

#train['median_price'] = train.median_price_x.combine_first(train.median_price_y)   

test.drop(['median_price_x','median_price_y'], inplace = True, axis = 1)                                      

In [85]:
#to do's:
#median/avg for non brand or category items
#submit kernel
#nn, sentiment/meaning, word2vec
#median or avg to replace worst 
#one hot encoding dummies for brand name in categories, test submission or before/after error in a new column? % diff?
#???

In [91]:
test_ids = np.array(test_id)
price_predictions = np.array(price_prediction)

predicted_cats_brands = pd.DataFrame({'test_id' : test_id, 'price_prediction' : price_predictions})

In [92]:
predicted_cats_brands.head(5)

,price_prediction,test_id
0,20.820801,206
1,6.021425,348
2,9.577393,829
3,22.908281,846
4,44.298839,931


In [93]:
test_final = pd.merge(test[['test_id','median_price']],predicted_cats_brands, how = 'left', on = 'test_id' )
test_final['final_pred'] = test_final.price_prediction.combine_first(test_final.median_price)

In [96]:
test_final.final_pred.fillna(value=10, inplace=True)

In [97]:
test_final.rename(columns={'final_pred':'price'}, inplace=True)

In [98]:
test_final[['test_id','price']].to_csv('/Users/Jpeteyy/Desktop/mercari_sub.csv', index = False)

In [99]:
test_final.head()

,test_id,median_price,price_prediction,price
0,0,NaN,11.909075,11.909075
1,1,NaN,14.132479,14.132479
2,2,NaN,22.520027,22.520027
3,3,NaN,10.367929,10.367929
4,4,NaN,8.098345,8.098345
